In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import hctmicrobiomemskcc.dataloading.dataloading as hct_data
import hctmicrobiomemskcc.tools.microbiotatools as hct_tools

# data generated from rotational_project1.jpynb
df1 = pd.read_csv('/Users/brian/Documents/2021Spring/Jonas_Rotation/hctmicrobiome/notebooks/deconstruction_exploration/df_deconstruction_ASV.csv')
df1 = df.iloc[:,1:] # remove first column for redundancy 

/Users/brian/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


NameError: name 'df' is not defined

In [ ]:
non_zero_ASV_patients = df1.groupby('PatientID').sum().astype(bool).sum(axis=0)
prevalence = 0.1
total_patients = len(df1.groupby('PatientID').sum())
prev_filter = non_zero_ASV_patients/total_patients > prevalence
ASVs_keep = prev_filter.index[prev_filter.values]


In [ ]:
temp1 = df1[["SampleID","PatientID","Consistency"]]
temp2 = df1[ASVs_keep]
df1 = pd.concat([temp1, temp2], axis=1)

In [ ]:
# I want to use the first 2 days in the deconstruction pattern to predict the 3rd. 
# subset 3rd row out
df_y = df1[df1.index % 3 == 0]  # Selects every 3rd raw starting from 0
df_y = df_y.drop(['SampleID', 'Consistency', 'Day'], axis = 1) # S3 ASV values 

df_temp = df1[df1.index % 3 != 0]  # excludes every 3rd row
df_temp = df_temp.drop(['SampleID', 'Consistency', 'Day'], axis = 1)
df_temp_S1 = df_temp[df_temp.index % 2 == 0]
df_temp_S2 = df_temp[df_temp.index % 2 != 0]
df_x = pd.merge(df_temp_S1, df_temp_S2, on  = 'PatientID')


a = df_x['PatientID'].unique() 
b = df_y['PatientID'].unique()
keeps = set(a) & set(b)

# keeping only patients in both df_x and df_y
df_y = df_y[df_y['PatientID'].isin(keeps)]
df_x = df_x[df_x['PatientID'].isin(keeps)]



In [ ]:
df_y.shape

In [147]:
Y_temp = df_y.drop("PatientID", axis = 1).to_numpy()
X_temp = df_x.drop("PatientID", axis = 1).to_numpy()


In [50]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest=train_test_split(X_temp, Y_temp, test_size=0.15)

In [ ]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

In [49]:
# get the dataset
 
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, kernel_initializer='he_uniform', activation = 'relu'))
	model.compile(loss='mse', optimizer='adam')
	return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
        results.append(mae)
    return results
 
# evaluate model
results = evaluate_model(xtrain, ytrain)
# summarize performance
print('MSE: %.3f (%.3f)' % (mean(results), std(results)))



>0.004
>0.005
>0.008
>0.004
>0.007
>0.006
>0.007
>0.007
>0.007
>0.005
>0.006
>0.005
>0.005
>0.006
>0.006
>0.005
>0.007
>0.006
>0.008
>0.006
>0.008
>0.005
>0.004
>0.006
>0.005
>0.005
>0.006
>0.006
>0.006
>0.006
MSE: 0.006 (0.001)


In [44]:
results

[7.685366153717041,
 7.823680400848389,
 8.424901962280273,
 7.777388095855713,
 7.245634078979492,
 7.7990899085998535,
 7.5624918937683105,
 11.259905815124512,
 9.15885066986084,
 8.289794921875,
 7.997190952301025,
 7.381680965423584,
 7.472104549407959,
 7.111973285675049,
 8.479877471923828,
 8.555052757263184,
 8.378199577331543,
 7.124639987945557,
 8.426131248474121,
 6.8342366218566895,
 7.994563102722168,
 8.843591690063477,
 8.327744483947754,
 7.85092830657959,
 8.792013168334961,
 7.835689067840576,
 7.141651630401611,
 7.740916728973389,
 7.755084991455078,
 9.43722915649414]

In [179]:
# use mlp for prediction on multi-output regression
from numpy import asarray
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
 
 
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, kernel_initializer='he_uniform', activation = 'relu'))
	model.compile(loss='mse', optimizer='adam', metrics = 'mse')
	return model


# load dataset
X = X_temp
Y = Y_temp
n_inputs, n_outputs = X.shape[1], Y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, Y, verbose=0, epochs=100)

# make a prediction for new data
row = X_temp[1]
newX = asarray([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])


Predicted: [0.         0.04985058 0.         0.05155992 0.         0.
 0.         0.         0.03713223 0.03149575 0.06486807 0.
 0.0121725  0.00887343 0.         0.         0.00619798 0.
 0.01229772 0.         0.         0.         0.03802477 0.
 0.         0.00950048 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.03891996 0.         0.02917193 0.
 0.         0.         0.         0.         0.         0.00045583
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0. 

In [181]:
model.fit(X, Y, verbose=1, epochs=500)

Epoch 1/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4327e-04 - mse: 1.4327e-04
Epoch 2/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4303e-04 - mse: 1.4303e-04
Epoch 3/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4313e-04 - mse: 1.4313e-04
Epoch 4/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4293e-04 - mse: 1.4293e-04
Epoch 5/500
16/16 [==============================] - 0s 989us/step - loss: 1.4272e-04 - mse: 1.4272e-04
Epoch 6/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4236e-04 - mse: 1.4236e-04
Epoch 7/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4264e-04 - mse: 1.4264e-04
Epoch 8/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4269e-04 - mse: 1.4269e-04
Epoch 9/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4167e-04 - mse: 1.4167e-04
Epoch 10/500
16/16 [==============================] - 0s 1ms/step - loss: 1.4162

16/16 [==============================] - 0s 1ms/step - loss: 1.2197e-04 - mse: 1.2197e-04
Epoch 160/500
16/16 [==============================] - 0s 2ms/step - loss: 1.2186e-04 - mse: 1.2186e-04
Epoch 161/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2198e-04 - mse: 1.2198e-04
Epoch 162/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2148e-04 - mse: 1.2148e-04
Epoch 163/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2133e-04 - mse: 1.2133e-04
Epoch 164/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2113e-04 - mse: 1.2113e-04
Epoch 165/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2105e-04 - mse: 1.2105e-04
Epoch 166/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2094e-04 - mse: 1.2094e-04
Epoch 167/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2083e-04 - mse: 1.2083e-04
Epoch 168/500
16/16 [==============================] - 0s 1ms/step - loss: 1.2

16/16 [==============================] - 0s 1ms/step - loss: 1.1150e-04 - mse: 1.1150e-04
Epoch 316/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1165e-04 - mse: 1.1165e-04
Epoch 317/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1155e-04 - mse: 1.1155e-04
Epoch 318/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1137e-04 - mse: 1.1137e-04
Epoch 319/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1148e-04 - mse: 1.1148e-04
Epoch 320/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1140e-04 - mse: 1.1140e-04
Epoch 321/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1146e-04 - mse: 1.1146e-04
Epoch 322/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1200e-04 - mse: 1.1200e-04
Epoch 323/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1108e-04 - mse: 1.1108e-04
Epoch 324/500
16/16 [==============================] - 0s 1ms/step - loss: 1.1

16/16 [==============================] - 0s 1ms/step - loss: 1.0609e-04 - mse: 1.0609e-04
Epoch 472/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0654e-04 - mse: 1.0654e-04
Epoch 473/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0621e-04 - mse: 1.0621e-04
Epoch 474/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0559e-04 - mse: 1.0559e-04
Epoch 475/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0558e-04 - mse: 1.0558e-04
Epoch 476/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0546e-04 - mse: 1.0546e-04
Epoch 477/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0570e-04 - mse: 1.0570e-04
Epoch 478/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0579e-04 - mse: 1.0579e-04
Epoch 479/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0562e-04 - mse: 1.0562e-04
Epoch 480/500
16/16 [==============================] - 0s 1ms/step - loss: 1.0

In [178]:
model.summary()

Model: "sequential_129"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_256 (Dense)            (None, 20)                13060     
_________________________________________________________________
dense_257 (Dense)            (None, 326)               6846      
Total params: 19,906
Trainable params: 19,906
Non-trainable params: 0
_________________________________________________________________
